## Настройки Colab

In [1]:
# Настройка пользователя (сделать один раз)
!git config --global user.email "nabludatellip@gmail.com"
!git config --global user.name "ProninPV"

In [2]:
from getpass import getpass

# 1. Безопасный ввод токена
GITHUB_TOKEN = getpass('Введите ваш GitHub Personal Access Token: ')

Введите ваш GitHub Personal Access Token: ··········


In [3]:
!git clone https://github.com/ProninPV/ml-regression_concrete-strength.git
%cd ml-regression_concrete-strength

Cloning into 'ml-regression_concrete-strength'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 455 (delta 240), reused 417 (delta 205), pack-reused 0 (from 0)
Receiving objects: 100% (455/455), 3.66 MiB | 6.76 MiB/s, done.
Resolving deltas: 100% (240/240), done.
/content/ml-regression_concrete-strength


In [47]:
%cd /content/ml-regression_concrete-strength
!git config pull.rebase false
!git pull origin tuning

/content/ml-regression_concrete-strength
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 346 bytes | 173.00 KiB/s, done.
From https://github.com/ProninPV/ml-regression_concrete-strength
 * branch            tuning     -> FETCH_HEAD
   476b0a9..87bcf07  tuning     -> origin/tuning
Updating 476b0a9..87bcf07
Fast-forward
 config/config.yaml | 4 ----
 1 file changed, 4 deletions(-)


In [5]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.5 MB/s eta 0:00:00


In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 5.5 MB/s eta 0:00:00


## 6.0 Импорты библиотек

In [7]:
import os
import yaml
import logging
import pickle
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from scipy.stats import ttest_ind
from typing import List, Any, Optional, Tuple, Dict, Union
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import psutil
from tqdm import tqdm
import gc
import optuna
import joblib

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [11]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [48]:
# В Colab проект клонируется в /content/
# Устанавливаем правильную рабочую директорию
project_root = Path('/content/ml-regression_concrete-strength')

# Определяем корень проекта
# cwd = Path().resolve()
# project_root = cwd.parent

# Добавляем корень проекта в sys.path (этого достаточно)
sys.path.append(str(project_root))

# Проверяем наличие конфиг файла
config_path = project_root / "config" / "config.yaml"
print(f"Looking for config at: {config_path}")

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing
from src.modeling import modeling

# Передаем путь явно
config = loader.load_config(config_path)
print("✅ Config loaded successfully!")

Looking for config at: /content/ml-regression_concrete-strength/config/config.yaml
✅ Config loaded successfully!


## 6.1. Загрузка данных

In [13]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train',
                                         config=config)

[⧗] Загружаю данные из: /content/ml-regression_concrete-strength/data/processed/eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [ ]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

In [14]:
# Загрузка отчета по экспериментам
modeling_result = modeling.load_sorted_modeling_report(config)
modeling_result

,outlier_strategy,model_name,model_type,dataset_size,mean_rmse,std_rmse,training_time_sec,memory_used_mb
0,['abnormal'],CatBoostRegressor,trees_models,770,4.2595,0.3446,13.49,0.44
1,['abnormal'],CatBoostRegressor,linear_models,770,4.2608,0.3868,11.55,12.38
2,"['gost_binar', 'combine']",CatBoostRegressor,linear_models,770,4.3196,0.4173,13.77,0.21
3,"['gost_binar', 'combine']",CatBoostRegressor,trees_models,770,4.3260,0.3906,14.72,0.03
4,['combine'],CatBoostRegressor,linear_models,770,4.3304,0.4210,11.68,0.15
...,...,...,...,...,...,...,...,...
105,['abnormal'],ElasticNet,trees_models,770,8.2662,0.4285,1.57,0.00
106,"['gost_binar', 'combine']",ElasticNet,linear_models,770,8.3014,0.4700,1.47,0.00
107,['gost_binar'],ElasticNet,linear_models,781,8.3183,0.5227,1.38,0.00
108,['abnormal'],ElasticNet,linear_models,770,8.3673,0.4635,1.29,0.00


In [55]:
# Определяем лучшую стратию
# best_strategy = modeling.get_best_model_strategy(modeling_result)
rf_row = modeling_result.iloc[26]

best_strategy = {
    'model': rf_row['model_name'],
    'model_type': rf_row['model_type'],
    'outlier_strategy': rf_row['outlier_strategy'],
    'rmse': rf_row['mean_rmse']
}

best_model = best_strategy['model']
best_model_type = best_strategy['model_type']
best_rmse = best_strategy['rmse']
best_outlier_strategy = eval(best_strategy['outlier_strategy'])

In [56]:
best_strategy

{'model': 'XGBRegressor',
 'model_type': 'linear_models',
 'outlier_strategy': "['gost_binar', 'combine']",
 'rmse': np.float64(4.5916)}

## 6.2. Предобработка данных

In [17]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

In [18]:
# Конфигурация для преобразования признаков
feature_config = {
    'trend_settings': {
        'names': ['Linear', 'Log', 'Sqrt']  # Только безопасные
    }
}

In [19]:
# Обрабатываем выбросы для лучшей стратегии
outlier_handler = feat_preprocessing.OutlierHandler(
    strategies=best_outlier_strategy,
    config=config,
    target_col=y_name
)

X_processed, y_processed = outlier_handler.fit_transform(X, y)

In [20]:
X_processed

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct,Outlier_age,Outlier_cement,Outlier_coarse_aggregate,Outlier_fine_aggregate,Outlier_fly_ash,Outlier_sp_c_pct,Outlier_w_c,Outlier_water,High_SP,High_FA
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,0.570745,0.000000,0,0,0,0,0,1,0,0,0,0
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,0.427699,0.007943,0,0,1,0,0,0,0,0,0,0
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,0.749600,0.022000,0,0,1,1,0,0,1,0,0,1
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,0.619355,0.000000,0,0,0,1,0,1,0,0,0,1
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,0.768743,0.032923,0,0,1,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,3,0.619355,0.000000,0,0,0,1,0,1,0,0,0,1
794,213.5,0.0,174.2,159.2,11.7,1043.6,771.9,100,0.745667,0.054801,0,0,0,0,0,1,1,0,1,0
795,304.8,0.0,99.6,196.0,9.8,959.4,705.2,28,0.643045,0.032152,0,0,1,0,0,1,0,0,1,0
797,288.0,192.0,0.0,192.0,0.0,932.0,717.8,28,0.666667,0.000000,0,0,1,0,0,1,0,0,0,0


## 6.2. Подбор гиперпараметров

#### Определение фнукций

In [21]:
def create_cv_strategy(config: Dict[str, Any]):
    """
    Создает стратегию кросс-валидации из конфига.
    """
    cv_config = config['cross_validation']
    if cv_config['strategy'] == 'RepeatedKFold':
        return RepeatedKFold(
            n_splits=cv_config['n_splits'],
            n_repeats=cv_config['n_repeats'],
            random_state=cv_config['random_state']
        )
    else:
        return KFold(n_splits=cv_config['n_splits'], shuffle=True, random_state=cv_config['random_state'])

In [22]:
class ProgressBarCallback:
    def __init__(self, n_trials: int):
        self.n_trials = n_trials
        self.pbar = tqdm(total=n_trials, desc="Optuna Trials", unit="trial")
        self.completed_trials = 0

    def __call__(self, study: optuna.Study, trial: optuna.Trial):
        # Обновляем только при завершении trial
        if trial.state.is_finished():
            self.completed_trials += 1
            self.pbar.update(1)
            self.pbar.set_postfix({
                'best_rmse': f"{study.best_value:.4f}" if study.best_value else "N/A",
                'completed': self.completed_trials
            })

        if self.completed_trials >= self.n_trials:
            self.pbar.close()

#### Catboost

In [ ]:
def create_catboost_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для CatBoost на основе конфига Optuna.
    """
    params = {}
    param_space = config['catboost_params']['search_space']

    # Динамически создаем параметры из search_space
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])
        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

    # Добавляем фиксированные параметры
    params.update(config['catboost_params']['fixed_params'])

    return params

In [ ]:
# Проверяем, что лучшая модель - CatBoost
if best_strategy['model'] != 'CatBoostRegressor':
    print(f"⚠️  Лучшая модель не CatBoost, а {best_strategy['model']}")

print("🎯 Начинаем тюнинг лучшей CatBoost модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_catboost_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    params['allow_writing_files'] = False

    # Создаем модель CatBoost с отключенным выводом
    params['verbose'] = False
    model = CatBoostRegressor(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"catboost_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ
best_params['verbose'] = False

best_model = CatBoostRegressor(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

In [ ]:
# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/catboostregression_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_catboostregression_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

#### RandomForest

In [23]:
def create_randomforest_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для RandomForest с проверкой совместимости.
    """
    params = {}
    param_space = config['randomforest_params']['search_space']

    # Обрабатываем параметры
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_categorical':
            params[param_name] = suggest_method(param_name, param_config['choices'])

    # ПОСЛЕ получения всех параметров проверяем конфликт
    if 'bootstrap' in params and 'max_samples' in params:
        if params['bootstrap'] is False and params['max_samples'] is not None:
            params['max_samples'] = None  # исправляем конфликт

    # Добавляем фиксированные параметры
    params.update(config['randomforest_params']['fixed_params'])

    return params

In [24]:
print("🎯 Начинаем тюнинг лучшей RandomForest модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_randomforest_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    # Создаем модель CatBoost с отключенным выводом
    params['verbose'] = False
    model = RandomForestRegressor(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"rf_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ
best_params['verbose'] = False

best_model = RandomForestRegressor(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

[I 2025-11-20 09:17:20,828] A new study created in memory with name: rf_tuning


🎯 Начинаем тюнинг лучшей RandomForest модели:
   Модель: RandomForestRegressor
   Стратегия выбросов: ['gost_binar', 'combine']
   Базовый RMSE: 4.9451
⏳ Запускаем Optuna...
RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)


  0%|          | 0/100 [00:00<?, ?it/s]


✅ Тюнинг завершен!
📊 Результаты:
   До тюнинга: 4.9451
   После тюнинга: 4.7907
   Улучшение: 0.1544 (3.1%)


In [25]:
# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/randomforest_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_randomforest_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

['models/tuning_models/randomforest_4_7907.pkl']

💾 Модель сохранена: models/tuning_models/randomforest_4_7907.pkl


['models/pipelines/pipeline_randomforest_4_7907.pkl']

💾 Пайплайн сохранен: models/pipelines/pipeline_randomforest_4_7907.pkl


#### Lightgbm

In [30]:
def create_lightgbm_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для LightGBM на основе конфига Optuna.
    """
    params = {}
    param_space = config['lightgbm_params']['search_space']

    # Обрабатываем параметры
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_categorical':
            params[param_name] = suggest_method(param_name, param_config['choices'])

    # Добавляем фиксированные параметры
    params.update(config['lightgbm_params']['fixed_params'])

    return params

In [35]:
print("🎯 Начинаем тюнинг лучшей RandomForest модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_lightgbm_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    model = LGBMRegressor(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"rf_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ

best_model = LGBMRegressor(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

🎯 Начинаем тюнинг лучшей RandomForest модели:
   Модель: LGBMRegressor
   Стратегия выбросов: ['gost_binar', 'combine']
   Базовый RMSE: 4.5113
⏳ Запускаем Optuna...
RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)


  0%|          | 0/100 [00:00<?, ?it/s]


✅ Тюнинг завершен!
📊 Результаты:
   До тюнинга: 4.5113
   После тюнинга: 4.1373
   Улучшение: 0.3740 (8.3%)


In [36]:
# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/lightgbm_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_lightgbm_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

['models/tuning_models/lightgbm_4_1373.pkl']

💾 Модель сохранена: models/tuning_models/lightgbm_4_1373.pkl


['models/pipelines/pipeline_lightgbm_4_1373.pkl']

💾 Пайплайн сохранен: models/pipelines/pipeline_lightgbm_4_1373.pkl


#### Ridge

In [49]:
def create_ridge_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для Ridge на основе конфига Optuna.
    """
    params = {}
    param_space = config['ridge_params']['search_space']

    # Обрабатываем параметры
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_categorical':
            params[param_name] = suggest_method(param_name, param_config['choices'])

    # Добавляем фиксированные параметры
    params.update(config['ridge_params']['fixed_params'])

    return params

In [50]:
print("🎯 Начинаем тюнинг лучшей RandomForest модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_ridge_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    model = Ridge(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"rf_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ

best_model = Ridge(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

🎯 Начинаем тюнинг лучшей RandomForest модели:
   Модель: Ridge
   Стратегия выбросов: ['iqr_remove']
   Базовый RMSE: 6.3750
⏳ Запускаем Optuna...
RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)


  0%|          | 0/100 [00:00<?, ?it/s]


✅ Тюнинг завершен!
📊 Результаты:
   До тюнинга: 6.3750
   После тюнинга: 6.5015
   Улучшение: -0.1265 (-2.0%)


In [51]:
# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/Ridge_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_Ridge_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

['models/tuning_models/Ridge_6_5015.pkl']

💾 Модель сохранена: models/tuning_models/Ridge_6_5015.pkl


['models/pipelines/pipeline_Ridge_6_5015.pkl']

💾 Пайплайн сохранен: models/pipelines/pipeline_Ridge_6_5015.pkl


#### XGBoost

In [57]:
def create_xgboost_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для XGBoost на основе конфига Optuna.
    """
    params = {}
    param_space = config['xgboost_params']['search_space']

    # Обрабатываем параметры
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])

        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])

        elif param_config['type'] == 'suggest_categorical':
            params[param_name] = suggest_method(param_name, param_config['choices'])

    # Добавляем фиксированные параметры
    params.update(config['xgboost_params']['fixed_params'])

    return params

In [58]:
print("🎯 Начинаем тюнинг лучшей RandomForest модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

print(cv_strategy)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_xgboost_params(trial, config)
    # params['task_type'] = 'GPU'  # ← ПРОСТО ДОБАВЛЯЕМ ЭТУ СТРОКУ

    model = XGBRegressor(**params)

    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )

    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        # n_jobs=1,
        error_score='raise'
    )

    rmse = -scores.mean()

    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_processed.shape[1])

    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 100

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"rf_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
# progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
# ДОБАВЬТЕ ЭТО ПЕРЕД optimize
optuna_logger = logging.getLogger('optuna')
optuna_logger.setLevel(logging.WARNING)  # ← Скрываем логи

# Запускаем оптимизацию
study.optimize(
    objective,
    n_trials=n_trials,
    # callbacks=[progress_callback],
    show_progress_bar=True,  # Оставляем встроенный бар
    # gc_after_trial=True
)

# Создаем лучшую модель
best_params = study.best_params
# best_params['task_type'] = 'GPU'  # ← ДОБАВЬТЕ ЗДЕСЬ

best_model = XGBRegressor(**best_params)

best_pipeline = modeling.create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipe = best_pipeline.fit(X_processed, y_processed)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

🎯 Начинаем тюнинг лучшей RandomForest модели:
   Модель: XGBRegressor
   Стратегия выбросов: ['gost_binar', 'combine']
   Базовый RMSE: 4.5916
⏳ Запускаем Optuna...
RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)


  0%|          | 0/100 [00:00<?, ?it/s]


✅ Тюнинг завершен!
📊 Результаты:
   До тюнинга: 4.5916
   После тюнинга: 4.0884
   Улучшение: 0.5032 (11.0%)


In [59]:
# Создаем директории если их нет
Path("models/tuning_models").mkdir(parents=True, exist_ok=True)
Path("models/pipelines").mkdir(parents=True, exist_ok=True)

# Форматируем имя файла (заменяем точку на подчеркивание для имени файла)
best_value_str = f"{study.best_value:.4f}".replace('.', '_')

# Сохраняем лучшую модель
model_filename = f"models/tuning_models/XGBRegressor_{best_value_str}.pkl"
joblib.dump(best_model, model_filename)
print(f"💾 Модель сохранена: {model_filename}")

# Сохраняем полный пайплайн
pipeline_filename = f"models/pipelines/pipeline_XGBRegressor_{best_value_str}.pkl"
joblib.dump(best_pipeline, pipeline_filename)
print(f"💾 Пайплайн сохранен: {pipeline_filename}")

['models/tuning_models/XGBRegressor_4_0884.pkl']

💾 Модель сохранена: models/tuning_models/XGBRegressor_4_0884.pkl


['models/pipelines/pipeline_XGBRegressor_4_0884.pkl']

💾 Пайплайн сохранен: models/pipelines/pipeline_XGBRegressor_4_0884.pkl


## Отправка на Githubm

In [60]:
!git status

On branch tuning
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	models/pipelines/pipeline_XGBRegressor_4_0884.pkl
	models/tuning_models/XGBRegressor_4_0884.pkl

nothing added to commit but untracked files present (use "git add" to track)


In [61]:
# 1. Добавляем файлы
!git add .
!git add models/tuning_models
!git add models/pipelines

# 2. Коммитим
!git commit -m "fit: add tuning XGboost regression"

[tuning 27fd17e] fit: add tuning lgbm regression
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 models/pipelines/pipeline_XGBRegressor_4_0884.pkl
 create mode 100644 models/tuning_models/XGBRegressor_4_0884.pkl


In [62]:
!git push https://{GITHUB_TOKEN}@github.com/ProninPV/ml-regression_concrete-strength.git tuning

Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 646.05 KiB | 5.38 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 3 local objects.
To https://github.com/ProninPV/ml-regression_concrete-strength.git
   3c4b163..27fd17e  tuning -> tuning
